In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3

In [2]:
# read poi from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)
    
names = pd.concat(places_names, axis=0)

names.to_csv('tianjin.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [9]:
names = pd.read_csv('tianjin.csv').name

In [10]:
names.head(2)

0    元记电气焊补胎钣金
1       元汇津通卸载
Name: name, dtype: object

In [11]:
# 'first_2w'
for lst in ['noimg_6w']:
    data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls')
    table = data.sheets()[0]          #通过索引顺序获取
    
    # get full-text tweets
    texts = []
    test_idx = 5 if lst == 'first_2w' else 6
    for i in range(1, table.nrows):
        texts.append(table.row(i)[test_idx].value)

    texts = pd.DataFrame(texts, columns=['text'])
    # texts.insert(1, 'places', pd.Series([[]]*len(texts)))
    
    # match all palces for all lines
    places = []
    sparse_matirx = []
    for i in tqdm(names.unique()): #[297604:]
        try:
            result = texts.text.str.contains(i)
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue
    
    # save match results with pickle
    print(len(places), len(sparse_matirx))
    with open('place2index_%s.pickle' % lst, 'wb') as f:
        pickle.dump([places, sparse_matirx], f)
        
    # print the most places in all tweets
    name2num = []
    for i,p in enumerate(places):
        name2num.append(sparse_matirx[i].sum())

    name2num = np.array(name2num)
    order_num = np.flip(np.argsort(name2num))
    np_places = np.array(places)
    numofprint = 5
    for p, n in zip(np_places[order_num][:numofprint], name2num[order_num][:numofprint]):
        print(p, ":", n)

  0%|                                                                                                                                                                                  | 0/700182 [00:00<?, ?it/s]/tmp/ipykernel_2117267/2163108270.py:20: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  result = texts.text.str.contains(i)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 700182/700182 [14:39<00:00, 796.42it/s]

2369 2369
天津 : 5493
我们 : 320
我们|WOMEN : 320
天津市 : 276
天津大学 : 253
